---
## **Лекция: Прореживание (прунинг) нейронных сетей**
### **Цель:** Изучить методы оптимизации нейронных сетей через удаление избыточных параметров.

### **Теоретическая часть**
#### **1. Основные концепции и терминология**
**Переобучение (Overfitting)** — явление, когда модель слишком точно подстраивается под обучающие данные и теряет способность к обобщению. Регуляризация включает методы борьбы с переобучением, включая L1/L2-регуляризацию и прореживание.

**Прунинг (Pruning)** — процесс удаления менее значимых элементов нейросети (весов, нейронов, слоев) для уменьшения размера модели и ускорения вычислений. Эмпирически доказано: нейросети содержат до 90% избыточных параметров.

#### **2. Виды прунинга**
**Неструктурированный прунинг** — удаление отдельных весов. Пример: magnitude-based pruning (удаление весов с наименьшими абсолютными значениями).

**Структурированный прунинг** — удаление целых блоков сети (нейронов, каналов свёрток). Эффективнее для аппаратной реализации.

**По времени применения:**
- До обучения (на случайной инициализации)
- Во время обучения (с L1-регуляризацией)
- После обучения (оптимизация готовой модели)

#### **3. Критерии значимости параметров**
- **Magnitude-based:** Веса с малыми абсолютными значениями считаются неважными.
- **Gradient-based:** Анализ влияния весов на градиент функции потерь.
- **Hessian-based:** Учёт кривизны функции потерь (OBD/OBS-методы).

#### **4. Связь с другими методами оптимизации**
- **Квантизация (Quantization):** Сокращение разрядности весов (например, float32 → int8).
- **Дистилляция знаний (Knowledge Distillation):** Передача знаний от большой модели к меньшей.
- **Низкоранговое разложение (Low-Rank):** Аппроксимация матриц весов.

#### **5. Историческая эволюция методов прунинга**
**1989-1993: Зарождение концепций**  
- Работа <NAME> *«Optimal Brain Damage»* ввела понятие значимости весов через анализ гессиана.  
- Эксперименты на мелких сетях (N<1000 параметров) показали возможность 10-кратного сжатия без потери точности.

**2015-2018: Эра глубокого обучения**  
- Magnitude pruning стал стандартом для ResNet (Han et al., 2015).  
- Структурированный прунинг для свёрточных сетей: удаление фильтров по L1-норме выходных каналов (Li et al., 2016).

**2020-2024: Нейросинтез и автоматизация**  
- Методы вроде *Neural Architecture Search* (NAS) интегрируют прунинг в процесс проектирования архитектур.  
- Техники динамического обновления масок (RigL) достигают 85% сжатия на Transformers.

#### **6. Математические основы прунинга**
**Формализация задачи:**  
Для модели с весами $W$ найти бинарную маску $M$, минимизирующую:  
$$
\mathcal{L}_{\text{prune}} = \mathcal{L}(W \odot M) + \lambda \|M\|_0 \to \min,
$$  
где $\|M\|_0$ — количество активных параметров.

**Типы регуляризации:**  
- **L0-аппроксимация:** Замена недифференцируемой L0-нормы на гладкую функцию:  
$$
\|M\|_0 \approx \sum \sigma(\beta w_i), \quad \sigma(z) = \frac{1}{1 + e^{-z}}.
$$
- **Стохастическое прореживание:** Моделирование маски как бернуллиевских случайных величин.

#### **7. Продвинутые алгоритмы прунинга**
**Hessian-ориентированные методы:**  
- **OBS (Optimal Brain Surgeon):** Учёт недиагональных элементов гессиана:  
$$
\Delta \mathcal{L} = \frac{1}{2} w_i^2 [H^{-1}]_{ii}^{-1}.
$$  
- **Критерий Тейлора:** Важность веса $w_i$ оценивается через:  
$$
I_i = |w_i \cdot \nabla_{w_i} \mathcal{L}|.
$$

**SNIP (Single-shot Network Pruning):**  
Ранжирование весов по чувствительности до обучения:  
$$
S_i = \left| w_i \cdot \frac{\partial \mathcal{L}}{\partial w_i} \right|.
$$

#### **8. Практические аспекты реализации**
**Обработка различных слоёв:**  
- **Conv-слои:** Удаление фильтров с нормой ниже порога $\theta = \mu - 2\sigma$.  
- **BatchNorm:** Учет параметров масштаба/сдвига при структурированном прунинге.
- **Skip-connections:** Запрет обрезки в residual-блоках для сохранения градиентов.

**Гиперпараметры:**  
- Итеративное прореживание с `final_sparsity=0.8`, `frequency=100` шагов.  
- Совместная оптимизация с learning rate decay (коэф. $\eta(t) = \eta_0 e^{-t/T}$).

#### **9. Анализ эффективности**
**Метрики:**  
- *Compression Ratio (CR):* $CR = \frac{\text{Исх. параметры}}{\text{Прунинг. параметры}}$  
- *Accuracy Drop (AD):* $AD = \text{Acc}_{\text{base}} - \text{Acc}_{\text{pruned}}$  
- *Теоретическое ускорение:* Рассчитывается через FLOPs reduction.

**Бенчмарки (ImageNet):**  
| Метод          | ResNet-50 (AD) | ViT-Base (AD) | CR  |
|----------------|----------------|---------------|-----|
| Magnitude      | 1.2%           | 3.8%          | 10x |
| SNIP           | 0.7%           | 2.1%          | 7x  |
| Variational    | 0.9%           | 1.9%          | 12x |

#### **10. Современные исследования и вызовы**
**Тенденции 2024:**  
- **Прунинг без данных:** Алгоритмы вроде *Data-Free Pruning* (DFP) используют синтетические данные.  
- **Кросс-модальное прореживание:** Единая маска для мультимодальных моделей (текст+изображение).

**Проблемы:**  
- Деградация качества при CR > 20x из-за нарушения manifold-структуры данных.  
- Неадаптивность масок к доменным сдвигам.

#### **11. Интеграция с другими методами**
**Квантование + Прунинг:**  
- 8-битное квантование после прунинга даёт дополнительное 4x сжатие (Google's QAT).

**Дистилляция:**  
- Учитель с 100M параметров → студент 20M + прунинг до 5M (DistilBERT).

### **Примеры кода (дополнение)**
**Iterative Pruning в PyTorch:**
```python
for epoch in range(10):
    # Обучение
    train_model(model, loader)
    
    # Прунинг 10% весов каждые 2 эпохи
    if epoch % 2 == 0:
        prune.global_unstructured(
            parameters=model.parameters(),
            pruning_method=prune.L1Unstructured,
            amount=0.1
        )
```

**Structured Pruning для Conv-слоёв:**
```python
# Фильтры с наименьшей L2-нормой
norms = torch.norm(conv.weight.data, p=2, dim=(1,2,3))
thresh = torch.kthvalue(norms, int(0.3 * len(norms))).values
mask = norms > thresh
conv.weight.data = conv.weight.data[mask]
```
---
### **Практическая часть**
#### **Пример 1: Magnitude Pruning в PyTorch**
```python
import torch
import torch.nn.utils.prune as prune

# Шаг 1: Создаём простую модель
model = torch.nn.Sequential(
    torch.nn.Linear(784, 256),  # Полносвязный слой
    torch.nn.ReLU(),
    torch.nn.Linear(256, 10)
)

# Шаг 2: Применяем прунинг к первому слою (удаляем 30% весов)
prune.l1_unstructured(module=model[0], name='weight', amount=0.3)

# Шаг 3: Проверяем разреженность
sparsity = 100 * float(torch.sum(model[0].weight == 0) / model[0].weight.nelement())
print(f'Разреженность после прунинга: {sparsity:.2f}%')
```
**Объяснение:**
1. `prune.l1_unstructured` удаляет веса с наименьшими L1-нормами.
2. Параметр `amount=0.3` задаёт долю удаляемых весов.
3. Веса заменяются на нули, но сохраняются в параметре `weight_orig`.

#### **Пример 2: Автоматическое прореживание через L1-регуляризацию**
```python
import tensorflow as tf

# Создаём модель с L1-регуляризацией
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu',
                          kernel_regularizer=tf.keras.regularizers.l1(0.01)),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam', loss='mse')
model.fit(x_train, y_train, epochs=10)

# Анализ нулевых весов
weights = model.layers[0].get_weights()[0]
zeros = tf.math.count_nonzero(weights == 0).numpy()
print(f'Занулено {zeros} весов из {weights.size}')
```
**Объяснение:**
1. L1-регуляризация добавляет штраф за большие значения весов, автоматически зануляя часть из них.
2. Коэффициент `0.01` регулирует силу регуляризации.

#### **Пример 3: Структурированный прунинг свёрточных каналов**
```python
from tensorflow_model_optimization.sparsity import keras as sparsity

# Настраиваем расписание прунинга
pruning_params = {
    'pruning_schedule': sparsity.PolynomialDecay(
        initial_sparsity=0.3,
        final_sparsity=0.7,
        begin_step=100,
        end_step=1000
    )
}

# Создаём модель с прунингом
pruned_model = tf.keras.Sequential([
    sparsity.prune_low_magnitude(
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        **pruning_params
    ),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10)
])
```
**Объяснение:**
1. `PolynomialDecay` плавно увеличивает степень прореживания от 30% до 70%.
2. Прунинг применяется к свёрточным фильтрам: удаляются целые каналы с малыми нормами.

---


---
**Литература и дополнительные материалы**
- [Оптимизация производитесности НС](https://www.iae.nsk.su/images/stories/4_Education/3_DisSovet/230530/2-%D0%A1%D0%92%D0%98%D0%A2%D0%9E%D0%92/dissertation_---%D0%A1%D0%92%D0%98%D0%A2%D0%9E%D0%92.pdf)
- [Машинное обучение](https://deepmachinelearning.ru/docs/Neural-networks/Regularization/Network-pruning)
- [ServerFlow](https://serverflow.ru/blog/stati/7-sposobov-uskorit-rabotu-iskusstvennogo-intellekta-i-sdelat-ego-bolee-lyegkim/)
- [Habr](https://habr.com/ru/articles/811221/)
- [Habr](https://habr.com/ru/articles/575520//)
- [Habr](https://habr.com/ru/articles/848306/)
